Fake therapist and schedule data

In [1]:
therapists = [
    {
        "name": "Dr. Jane Smith",
        "specialization": "anxiety, academic stress",
        "availability": "Mondays and Wednesdays, 2-4 PM",
        "contact": "jsmith@email.com, 555-1234",
        "description": "Dr. Jane Smith specializes in anxiety and academic stress. Available Mondays and Wednesdays, 2-4 PM. Contact: jsmith@email.com, 555-1234."
    },
    {
        "name": "Dr. John Doe",
        "specialization": "depression, relationships",
        "availability": "Tuesdays and Thursdays, 10 AM - 12 PM",
        "contact": "jdoe@email.com, 555-5678",
        "description": "Dr. John Doe specializes in depression and relationships. Available Tuesdays and Thursdays, 10 AM - 12 PM. Contact: jdoe@email.com, 555-5678."
    },
    {
        "name": "Dr. Emily Brown",
        "specialization": "stress management, self-esteem",
        "availability": "Fridays, 1-3 PM",
        "contact": "ebrown@email.com, 555-9012",
        "description": "Dr. Emily Brown specializes in stress management and self-esteem. Available Fridays, 1-3 PM. Contact: ebrown@email.com, 555-9012."
    }
]

classes = [
    {
        "class_name": "Math 101",
        "instructor": "Prof. Johnson",
        "schedule": "Tuesdays and Thursdays, 10-11:30 AM, Room 202",
        "exam_dates": "May 15",
        "assignments": "Assignment 1 due May 10",
        "description": "Math 101 with Prof. Johnson, Tuesdays and Thursdays, 10-11:30 AM, Room 202. Exam on May 15, Assignment 1 due May 10."
    },
    {
        "class_name": "History 202",
        "instructor": "Prof. Lee",
        "schedule": "Mondays and Wednesdays, 1-2:30 PM, Room 305",
        "exam_dates": "May 12",
        "assignments": "Essay due May 8",
        "description": "History 202 with Prof. Lee, Mondays and Wednesdays, 1-2:30 PM, Room 305. Exam on May 12, Essay due May 8."
    },
    {
        "class_name": "Biology 150",
        "instructor": "Prof. Carter",
        "schedule": "Wednesdays and Fridays, 9-10:30 AM, Room 101",
        "exam_dates": "May 10",
        "assignments": "Lab report due May 5",
        "description": "Biology 150 with Prof. Carter, Wednesdays and Fridays, 9-10:30 AM, Room 101. Exam on May 10, Lab report due May 5."
    }
]

Set up

In [ ]:
!pip install openai faiss-cpu numpy


In [3]:
import openai
import faiss
import numpy as np

openai.api_key = ""


Embedding and FAISS

In [4]:
def embed_documents(docs):
    response = openai.embeddings.create(input=docs, model="text-embedding-ada-002")
    return np.array([r.embedding for r in response.data], dtype='float32')

# Embed only descriptions
therapist_descriptions = [t["description"] for t in therapists]
therapist_embeddings = embed_documents(therapist_descriptions)
class_descriptions = [c["description"] for c in classes]
class_embeddings = embed_documents(class_descriptions)

# FAISS
embedding_dim = therapist_embeddings.shape[1]
therapist_index = faiss.IndexFlatL2(embedding_dim)
therapist_index.add(therapist_embeddings)
class_index = faiss.IndexFlatL2(embedding_dim)
class_index.add(class_embeddings)



Retrieval 

In [5]:
def get_relevant_therapist(query, k=1):
    query_embedding = embed_documents([query])
    distances, indices = therapist_index.search(query_embedding, k)
    return [therapists[i] for i in indices[0]]

def get_relevant_class(query, k=1):
    query_embedding = embed_documents([query])
    distances, indices = class_index.search(query_embedding, k)
    return [classes[i] for i in indices[0]]


Demo

In [6]:
query = "I am feeling depression about next week's history finals."
relevant_therapist = get_relevant_therapist(query)
relevant_class = get_relevant_class(query)

print("Relevant Therapist:", relevant_therapist[0]["name"])
print("Therapist Details:", relevant_therapist[0]["description"])
print("Relevant Class:", relevant_class[0]["class_name"])
print("Class Details:", relevant_class[0]["description"])

Relevant Therapist: Dr. John Doe
Therapist Details: Dr. John Doe specializes in depression and relationships. Available Tuesdays and Thursdays, 10 AM - 12 PM. Contact: jdoe@email.com, 555-5678.
Relevant Class: History 202
Class Details: History 202 with Prof. Lee, Mondays and Wednesdays, 1-2:30 PM, Room 305. Exam on May 12, Essay due May 8.
